In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### Gemini 3 and Gemini 2.5 Response Format 

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage

gemini3 = "gemini-3-pro-preview"
# gemini2 = "gemini-2.5-pro"
gemini2 = "gemini-2.5-flash"

system_msg = SystemMessage("You are a helpful assistant.")

query = "Explain the theory of relativity in simple terms."
messages = [system_msg, HumanMessage(query)]


In [4]:
model = ChatGoogleGenerativeAI(model=gemini3)
response = model.invoke(messages)

KeyboardInterrupt: 

In [ ]:
response

In [ ]:
response.text

In [5]:
model = ChatGoogleGenerativeAI(model=gemini2)
response = model.invoke(messages)

In [6]:
response

AIMessage(content='Of course! Let\'s break down Einstein\'s theory of relativity into its two parts, using simple analogies.\n\nThe big idea behind all of it is that **space and time are not separate and constant**. Instead, they are linked together in a single fabric called **spacetime**. What you experience as time and space depends on your motion and the gravity around you.\n\n---\n\n### Part 1: Special Relativity (The "Speed" Part)\n\nImagine you\'re driving on a highway. Special Relativity deals with what happens when you move at a **constant speed**. It\'s built on two simple but mind-bending ideas:\n\n1.  **The Laws of Physics are the Same for Everyone.** If you\'re on a perfectly smooth airplane with no windows, you can\'t tell if you\'re moving or standing still. If you throw a ball up, it comes straight back down, just like it would on the ground. Everything works the same.\n\n2.  **The Speed of Light is a Cosmic Speed Limit.** This is the weird one. The speed of light is the

In [7]:
response.content
response.content_blocks
response.usage_metadata
response.response_metadata


{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.5-pro',
 'safety_ratings': [],
 'model_provider': 'google_genai'}

### Streaming

In [16]:
model = ChatGoogleGenerativeAI(model=gemini2)

query = "Write a short story about the earth and the moon."
for chunk in model.stream(query):
    print(chunk.content, end="", flush=True)

In the cosmic nursery, amidst swirling dust and nascent stars, Terra was born. She was a vibrant swirl of greens and blues, already humming with the promise of life, a restless energy thrumming beneath her nascent crust. But she was not alone. From the same swirling chaos, a smaller, paler sibling emerged – Luna.

Terra felt an immediate, irresistible tug towards him, a gravitational embrace that settled them into a perpetual dance. Luna, in turn, felt bound to her, a constant pull that kept him orbiting, a silent sentinel in the vast, indifferent dark.

In the early eons, Terra was a tempest. Volcanoes roared, oceans boiled, and lightning cracked across her primordial skies. Luna watched it all, his pockmarked face reflecting her fiery youth, absorbing her wild, untamed light. He was her first, most loyal observer, the quiet witness to her chaotic birth.

As Terra cooled, life began to stir. Tiny, single-celled organisms bloomed into iridescent carpets, then into forests that clawed a

### Multimodal usage

In [17]:
model = ChatGoogleGenerativeAI(model=gemini3)

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the image provided."},
        {
            "type": "image",
            "url": "https://www.shutterstock.com/image-vector/vector-cute-baby-panda-cartoon-600nw-2427356853.jpg",
        },
    ]
)
response = model.invoke([message])

In [18]:
response.content
response.text

'This is a cute, cartoon-style illustration of a giant panda. Here are the details:\n\n*   **Subject:** A baby panda sitting upright and facing forward.\n*   **Face:** The panda has a large, round head with big, expressive eyes that feature white sparkles. The eyes are surrounded by classic black patches. It has a small black nose and a smiling mouth with a pink tongue visible. The round ears are black on the outside with pink centers.\n*   **Body:** The body is chubby with black fur on the arms, legs, and back, and a round white belly. It is in a sitting position with its rear paws facing forward, revealing pink paw pads (one large pad and three smaller toe pads on each foot).\n*   **Background:** Behind the panda, there are stylized clumps of tall, green grass blades fanning out. The rest of the background is plain white.\n*   **Style:** The image uses clean lines, bright colors, and smooth shading, characteristic of vector art or digital clipart designed to look adorable ("kawaii").

In [19]:
## Reading media from local file and encoding to base64
## Now use smaller model for faster response

## image mime type example
# mime_type = "image/png"

## pdf mime type example
# mime_type = "application/pdf", type = "file"

## audio mime type example
# mime_type = "audio/mpeg", type = "audio"

import base64

image_bytes = open("data/images/panda.png", "rb").read()
bytes_base64 = base64.b64encode(image_bytes).decode("utf-8")

mime_type = "image/png"

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the image provided."},
        {
            "type": "image",
            "base64": bytes_base64,
            "mime_type": mime_type,
        },
    ]
)

model = ChatGoogleGenerativeAI(model=gemini2)
response = model.invoke([message])

In [20]:
response

AIMessage(content='The image is a cheerful and adorable cartoon illustration featuring a baby panda sitting amidst vibrant green grass on a plain white background.\n\nThe central figure is the panda, rendered in a cute, "chibi" style with a proportionally large head and a small, round body. It has the classic black and white fur pattern: a white face and belly, with black fur forming prominent eye patches, round ears, arms, and legs.\n\nThe panda\'s face is particularly expressive. It has large, dark, sparkling eyes with small white highlights that give them a friendly, innocent look. A subtle hint of eyelashes can be seen on the outer edge of its eyes. Its nose is small, black, and oval-shaped, situated above a wide, open smile that reveals a tiny pink tongue. The ears are round, black on the outside, and have a soft pink inner lining.\n\nThe panda is in a seated position, its plump body resting on the ground. Its black paws are visible, showcasing light pink paw pads, adding to its c

In [21]:
pdf_bytes = open("data/rag-data/apple/apple 10-q q1 2024.pdf", "rb").read()
pdf_base64 = base64.b64encode(pdf_bytes).decode("utf-8")

mime_type = "application/pdf"

message = HumanMessage(
    content=[
        {"type": "text", "text": "Summarize the key financial highlights from this quarterly report."},
        {
            "type": "file",
            "base64": pdf_base64,
            "mime_type": mime_type,
        },
    ]
)
response = model.invoke([message])

In [22]:
response.text

"Apple Inc.'s quarterly report for the three months ended March 30, 2024 (Q2 2024) reveals a mixed financial picture with a slight decline in overall revenue and net income, offset by strong Services growth and significant capital returns to shareholders.\n\nHere are the key financial highlights:\n\n*   **Total Net Sales:** Decreased by 4% to **$90.75 billion** in Q2 2024, down from $94.84 billion in Q2 2023.\n*   **Net Income:** Slightly decreased by 2% to **$23.64 billion** in Q2 2024, compared to $24.16 billion in Q2 2023.\n*   **Diluted Earnings Per Share (EPS):** Remained **flat at $1.53**, primarily due to the impact of share repurchases offsetting the decline in net income.\n*   **Gross Margin Percentage:** Improved significantly to **46.6%** in Q2 2024 from 44.3% in Q2 2023, driven by a favorable Services mix.\n*   **Segment Performance:**\n    *   **Services Revenue:** Continued its strong growth trajectory, increasing by 14% to **$23.87 billion** from $20.91 billion in Q2 202

In [29]:
response.content
response.content_blocks
response.usage_metadata
response.response_metadata

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.5-flash',
 'safety_ratings': [],
 'model_provider': 'google_genai'}

### Tool Calling

In [ ]:
# first show ollama web_search tool
# then build weather tool

from scripts import base_tools

In [ ]:
response = base_tools.web_search.invoke({'query': 'what is the latest US stock market updates?'})
# response

In [ ]:
response = base_tools.get_weather.invoke({'location': 'Mumbai'})
# response

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)
model_with_tools = model.bind_tools([base_tools.web_search, base_tools.get_weather])

In [ ]:
model_with_tools
response = model_with_tools.invoke("what is the weather in Mumbai today? and latest news on US stock market?")

In [ ]:
response

### Thinking support

https://ai.google.dev/gemini-api/docs/thinking

With reasoning models, you have the option to adjust the number of internal thinking tokens used (thinking_budget) or to disable thinking altogether.


In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2,
                               thinking_budget=1024,
                               include_thoughts=True)

query = "explain the theory of relativity in simple terms."
response = model.invoke(query)

In [ ]:
print(response)
# print(response.text)

response.content_blocks

response

In [ ]:
model_with_tools = model.bind_tools([base_tools.web_search, base_tools.get_weather])

response = model_with_tools.invoke(query)


In [ ]:
response.content_blocks
response

### Built-in tools

Google Gemini supports a variety of built-in tools, which can be bound to the model in the usual way.

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)

#  base_tools.get_weather -> function calling with tool use is unsupported.
model_with_tools = model.bind_tools([{"google_search": {}}, {"code_execution": {}}])

In [ ]:
query = "When is the next total solar eclipse in the US and what is 3 + 2?"
response = model_with_tools.invoke(query)

In [ ]:
print(response.text)

In [ ]:
response.response_metadata

### Context Caching

- https://ai.google.dev/gemini-api/docs/caching?hl=en&lang=python#pdfs_1
- https://ai.google.dev/gemini-api/docs/pricing

In [33]:
import time
from google import genai
from google.genai.types import CreateCachedContentConfig, Content, Part

client = genai.Client()

In [34]:
file_paths = [
    "data/rag-data/apple/apple 10-q q1 2024.pdf",
    "data/rag-data/apple/apple 10-q q2 2024.pdf"
]

uploaded_files = []
for path in file_paths:
    file = client.files.upload(file=path)
    while file.state.name == "PROCESSING":
        time.sleep(2)
        file = client.files.get(name=file.name)
    uploaded_files.append(file)

In [35]:
parts = []
for f in uploaded_files:
    part = Part.from_uri(file_uri=f.uri, mime_type=f.mime_type)
    parts.append(part)

contents = [
    Content(
        role="user",
        parts=parts,
    )
]

cache = client.caches.create(
    model=gemini2,
    config=CreateCachedContentConfig(
        display_name="Apple Q1 Q2 2024 Reports",
        system_instruction="You are a financial analyst. Use these Apple quarterly reports to answer questions.",
        contents=contents,
        ttl="300s",
    ),
)

In [ ]:
llm = ChatGoogleGenerativeAI(
    model=gemini2,
    cached_content=cache.name,
)

response = llm.invoke("Compare the revenue growth between Q1 and Q2 2024.")

In [41]:
# print(response.text)
from IPython.display import Markdown, display
display(Markdown(response.text))

Based on the provided Apple quarterly reports:

The first report is for the fiscal quarter ended **March 30, 2024**, which is Apple's **second fiscal quarter (Q2)**. Apple's fiscal Q1 would typically end in December.

For the **three months ended March 30, 2024 (fiscal Q2)**, Apple's total operating expenses were **$14,371 million**.

(Source: Apple Inc. Q2 2024 Form 10-Q, Condensed Consolidated Statements of Operations, Page 4)

In [39]:
response.usage_metadata

{'input_tokens': 14482,
 'output_tokens': 1568,
 'total_tokens': 16050,
 'input_token_details': {'cache_read': 14465},
 'output_token_details': {'reasoning': 975}}

In [40]:
response = llm.invoke("What were the total operating expenses in Q1?")

In [42]:
display(Markdown(response.text))

Based on the provided Apple quarterly reports:

The first report is for the fiscal quarter ended **March 30, 2024**, which is Apple's **second fiscal quarter (Q2)**. Apple's fiscal Q1 would typically end in December.

For the **three months ended March 30, 2024 (fiscal Q2)**, Apple's total operating expenses were **$14,371 million**.

(Source: Apple Inc. Q2 2024 Form 10-Q, Condensed Consolidated Statements of Operations, Page 4)

In [43]:
response.usage_metadata

{'input_tokens': 14476,
 'output_tokens': 1068,
 'total_tokens': 15544,
 'input_token_details': {'cache_read': 14465},
 'output_token_details': {'reasoning': 941}}

### Structured Output

In [ ]:
# Use weather tool for the sample structred output
# fields -> location:str, date:str, temperature:str, condition:str

from pydantic import BaseModel

class WeatherOutput(BaseModel):
    location: str
    date: str
    temperature: str
    condition: str

model = ChatGoogleGenerativeAI(model=gemini2)
model_with_tools = model.bind_tools([base_tools.get_weather])

structured_model = model_with_tools.with_structured_output(WeatherOutput)

In [ ]:

response = structured_model.invoke("what is the weather in Mumbai today?")

In [ ]:
response